In [1]:
import requests
from datetime import datetime

# --------- Helper: Geocoding ----------
def fetch_coordinates(location_name: str):
    url = "https://geocoding-api.open-meteo.com/v1/search"
    params = {
        "name": location_name,
        "count": 1
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    if "results" not in data or not data["results"]:
        raise ValueError(f"Location not found: {location_name}")

    result = data["results"][0]
    return {
        "location_name": result["name"],
        "country": result.get("country"),
        "latitude": result["latitude"],
        "longitude": result["longitude"]
    }


# --------- Helper: Weather ----------
def fetch_hourly_weather(latitude: float, longitude: float):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m,wind_speed_10m,relative_humidity_2m",
        "past_days": 1,
        "timezone": "UTC"
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()["hourly"]


# --------- Main Ingestion Logic ----------
def ingest_weather_for_location(location_name):
    records = []


    # 1. Get coordinates
    location = fetch_coordinates(location_name)

    # 2. Get weather data
    hourly_data = fetch_hourly_weather(
        location["latitude"],
        location["longitude"]
    )

    times = hourly_data["time"]
    temperatures = hourly_data["temperature_2m"]
    wind_speeds = hourly_data["wind_speed_10m"]
    humidities = hourly_data["relative_humidity_2m"]

    # 3. Flatten to records (ONE record per hour per location)
    for i in range(len(times)):
        record = {
            "timestamp": datetime.fromisoformat(times[i]),
            "location_name": location["location_name"],
            "country": location["country"],
            "latitude": location["latitude"],
            "longitude": location["longitude"],
            "temperature": temperatures[i],
            "wind_speed": wind_speeds[i],
            "humidity": humidities[i]

        }
        records.append(record)

    return records



In [1]:
import requests

# נתוני דוגמה גולמיים
test_data = [{
    "timestamp": "2026-01-21T10:00:00",
    "location_name": "London",
    "country": "United Kingdom",
    "latitude": 51.5,
    "longitude": -0.12,
    "temperature": 26.5, # אמור להפוך ל-hot
    "wind_speed": 15.0,  # אמור להפוך ל-windy
    "humidity": 80
}]

# שליחה ל-Service B
response = requests.post("http://localhost:8002/clean", json=test_data)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

ConnectionError: HTTPConnectionPool(host='localhost', port=8002): Max retries exceeded with url: /clean (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002E2AFCF5BE0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:

# this triggers the whole pipeline to ingest weather for London and write to mysql
data = ingest_weather_for_location("London")
data

[{'timestamp': datetime.datetime(2026, 1, 19, 0, 0),
  'location_name': 'London',
  'country': 'United Kingdom',
  'latitude': 51.50853,
  'longitude': -0.12574,
  'temperature': 8.6,
  'wind_speed': 3.9,
  'humidity': 83},
 {'timestamp': datetime.datetime(2026, 1, 19, 1, 0),
  'location_name': 'London',
  'country': 'United Kingdom',
  'latitude': 51.50853,
  'longitude': -0.12574,
  'temperature': 8.2,
  'wind_speed': 3.4,
  'humidity': 84},
 {'timestamp': datetime.datetime(2026, 1, 19, 2, 0),
  'location_name': 'London',
  'country': 'United Kingdom',
  'latitude': 51.50853,
  'longitude': -0.12574,
  'temperature': 8.1,
  'wind_speed': 3.1,
  'humidity': 84},
 {'timestamp': datetime.datetime(2026, 1, 19, 3, 0),
  'location_name': 'London',
  'country': 'United Kingdom',
  'latitude': 51.50853,
  'longitude': -0.12574,
  'temperature': 8.1,
  'wind_speed': 2.6,
  'humidity': 86},
 {'timestamp': datetime.datetime(2026, 1, 19, 4, 0),
  'location_name': 'London',
  'country': 'United K